# 地震波数値シミュレーション入門

このノートブックでは，弾性体の運動方程式と線形弾性体の構成関係式を前提として，もっとも簡単な1次元の地震波伝播問題をスタガードグリッド差分法による数値シミュレーションを体験します．以下では座学と数値計算の実践や可視化が繰り返されますので，まず必要となるPythonライブラリのインポートをしておきましょう．その後，文章部分を学び，その下にあるコードを順に実行していくことで，地震波伝播の数値シミュレーションがどういうものかを体験してもらえるようになっています．

なお，この一連のノートブックでは，数値計算のためのライブラリとして [numpy](https://numpy.org) が，可視化ツールとして [PyGMT](https://www.pygmt.org/latest/) がそれぞれ使われています．NumPyについては多くの環境でインストール済みだと思います．PyGMTについては，たとえば[ここ](https://tktmyd.github.io/pygmt-howto-jp/install.html)にminiforgeを用いた簡易な環境構築の例があります．ライブラリの読み込みは，必要に応じてノートブックごとに，それぞれの先頭で行います．

## はじめに：弾性論と数値シミュレーション

地震波が震源から固体地球内部を広がっていく **地震波動伝播** の問題は，**弾性体の運動方程式**

$$
\rho \frac{\partial^2 u_i}{\partial t^2} = \sum_{j=1}^3 \frac{\partial \sigma_{ij}}{\partial x_j} + f_i
$$

と，応力テンソルとひずみテンソルとを関係づける **構成関係式** （ここでは**線形等方弾性体**を仮定する）

$$
\sigma_{ij} = \delta_{ij} \lambda \sum_{k=1}^3 \varepsilon_{kk} + 2 \mu \varepsilon_{ij}
$$

およびひずみテンソルの定義

$$
\varepsilon_{ij} = \frac{1}{2} \left( \frac{\partial u_i}{\partial x_j} + \frac{\partial u_j}{\partial x_i} \right)
$$

によって記述されます．ここで$\rho$は質量密度，$u_i$は弾性体の変位ベクトル，$f_i$は体積力のそれぞれ第$i$成分，$\sigma_{ij}$ と $\varepsilon_{ij}$は応力およびひずみテンソルの第$i,j$成分，$\lambda$と$\mu$はLaméの係数で，$P$波と$S$波の速さと

$$
\alpha = \sqrt{\frac{\lambda+2\mu}{\rho}}, \quad
\beta = \sqrt{\frac{\mu}{\rho}}
$$

という関係にあります．

地震（断層運動）は 体積力 $\boldsymbol{f}$ に相当する項で（ダブルカップル型の震源として）表され，地球内部の不均質構造は質量密度 $\rho$ と $\lambda$と$\mu$はLaméの係数 $\lambda$ と $\mu$（あるいは等価なことですが，地震波速度 $\alpha$ と $\beta$）が位置によって異なる値を取る，すなわち場所 $\boldsymbol{x}$ の関数であることによって表現されます．

このように，震源と地球内部構造とが与えられたとき，運動方程式と構成関係式を連立偏微分方程式としてもし**解く**ことができれば，すなわち$\boldsymbol{u}(\boldsymbol{x},t)$ のように場所 $\boldsymbol{x}$ と時間 $t$ の関数として弾性体の変位ベクトルを求めることができれば，与えられた震源によって既知の地球内部構造を伝わる地震波が完全に再現できた，ということになります．

しかし，弾性体の運動方程式はかなり複雑な方程式で，たとえば地球内部構造が完全に均質（密度やLaméの係数が一定）のようなきわめて限られた条件でしか，この方程式を解析的に解くことはできません．そこで，ここでは微分方程式に対してある **近似** を施すことで，この問題をコンピュータで直接答えを求められるような代数的な（四則演算による）計算のみによって解く，ということを考えます．このようなアプローチを一般に **数値シミュレーション** とか **数値計算** と呼びます．

## 問題設定と基礎方程式

いきなり現実世界の3次元空間における地震波のシミュレーションは，アルゴリズムも煩雑になりますし，大規模な計算機も必要になってしまいます．
そこでここでは，まず運動方程式と構成関係式を数値計算に都合のよい形に変形し，その次に，特別な場合として「1次元の方程式」を作ります．それに対して数値シミュレーションの技法を当てはめていくことにしましょう．

まずは，運動方程式について，変位のかわりに速度 $v_i = \partial u_i / \partial t$ を独立変数とします．もう一つ，構成関係式にはひずみの定義を代入してしまい，そのうえで両辺を1階時間で微分します．

運動方程式については，加速度ベクトル成分を変位速度で表すのですから，変位の2階微分のかわりに速度の1階微分を用いて

$$
\rho \frac{\partial v_i}{\partial t} = \sum_{j=1}^3 \frac{\partial \sigma_{ij}}{\partial x_j} + f_i
$$

となります．一方，構成関係式にひずみテンソルの定義を代入し，さらに時間微分すると，

$$
\frac{\partial \sigma_{ij}}{\partial t} = \delta_{ij} \lambda \sum_{k=1}^3 \frac{\partial v_k}{\partial x_k} + \mu \left( \frac{\partial v_i}{\partial x_j} + \frac{\partial v_j}{\partial x_i}  \right)
$$

が得られました．このように，変位速度と応力の時間微分に関する連立方程式を用いた表現のことを， **速度・応力型**の方程式と呼びます．運動方程式は左辺が変位速度の時間微分で右辺が応力の空間微分，構成関係式は左辺が応力の時間微分で右辺が変位速度の空間微分，と，きわめて対称性の高い形になっており，数値シミュレーションをするうえで解きやすいのです．

さらにここでは，1次元近似をします．空間微分のうち$x_1$微分以外はすべて$0$としてしまうと，以下の3つの組の方程式が得られます．

$$
\begin{align}
    &\rho \frac{\partial v_1}{\partial t} = \frac{\partial \sigma_{11}}{\partial x_1} , \quad \frac{\partial \sigma_{11}}{\partial t} = \left(\lambda+ 2\mu \right)\frac{\partial v_1}{\partial x_1}
    \\
    &\rho \frac{\partial v_2}{\partial t} = \frac{\partial \sigma_{12}}{\partial x_1} , \quad \frac{\partial \sigma_{12}}{\partial t} = \mu \frac{\partial v_2}{\partial x_1}    
    \\
    &\rho \frac{\partial v_3}{\partial t} = \frac{\partial \sigma_{13}}{\partial x_1} , \quad \frac{\partial \sigma_{13}}{\partial t} = \mu \frac{\partial v_3}{\partial x_1}    
\end{align}
$$

それぞれの式は，運動方程式1本と構成関係式1本の組で， **閉じた**（すなわち原理的には他の成分の式を用いずに解くことができる）形になっています．しかも，3本とも形式的には同じで，


$$
\rho \frac{\partial v}{\partial t} = \frac{\partial \sigma}{\partial x}, \quad
\frac{\partial \sigma}{\partial t} = G \frac{\partial v}{\partial x}
$$

という形をしています．$G$は**剛性率**に相当するもので，$\lambda+2\mu$ もしくは $\mu$ の値を取ります．また，空間方向は1方向しか現れないため，$x_1$のかわりに単に$x$と表記しました．ここではこれを1次元の方程式系，として受け入れましょう．

## 1次元地震波のスタガードグリッド差分

それでは，運動方程式と構成関係式

$$
\rho \frac{\partial v}{\partial t} = \frac{\partial \sigma}{\partial x}, \quad
\frac{\partial \sigma}{\partial t} = G \frac{\partial v}{\partial x}
$$

をスタガードグリッド差分法で表現してみましょう．ここでは簡単のため，密度$\rho$と剛性率$G$は定数であるとし，速度$v$と応力$\sigma$の変数を，それぞれ時間微分を時間ステップ$\Delta t$の，空間微分を空間グリッド間隔 $\Delta x$の，それぞれ中心差分で離散化していきます．

まずは先の例にならって空間微分を空間差分に置き換えてみましょう．
構成関係式と運動方程式の右辺の微分を差分に書き換えると，それぞれ

$$
\begin{align}
&\frac{\partial \sigma(x,t)}{\partial t} = G \frac{v(x+\Delta x/2,t)-v(x-\Delta x / 2, t)}{\Delta x}
\\
&\rho \frac{\partial v(x,t)}{\partial t} = \frac{\sigma(x+\Delta x/2,t)-\sigma(x-\Delta x / 2, t)}{\Delta x}
\end{align}
$$

と表されそうです．しかし，この組み合わせを直接コンピュータで解こうとすると困ったことがあります．構成関係式のほうに現れる応力 $\sigma$ の位置は $x$，変位速度 $v$の位置は $x\pm \Delta x/2$ なのに対して，運動方程式の位置は 応力が  $x\pm \Delta x/2$，変位速度が $x$ と，あべこべなのです．これでは，離散的な位置における変位速度と応力をうまく連立方程式として解くことができません．

そこで，構成関係式の位置を全体に $-\Delta x/2$ だけ平行移動してみましょう．そうすると，


$$
\begin{align}
&\frac{\partial \sigma(x-\Delta x/2,t)}{\partial t} = G \frac{v(x,t)-v(x-\Delta x, t)}{\Delta x}
\\
&\rho \frac{\partial v(x,t)}{\partial t} = \frac{\sigma(x+\Delta x/2,t)-\sigma(x-\Delta x / 2, t)}{\Delta x}
\end{align}
$$

となり，位置の対応関係が改善しました．

このうえで，$x$軸を間隔 $\Delta x$で等間隔に分割し，**グリッド位置** を

$$x_i = x_0 + i \Delta x$$

と定義します．ただし $x_0$ は座標系原点の基準位置，$x_i$ は $i$番目のグリッドにおける位置です．さらに，（ちょっと変則的ですが） 

$$
v(x_i,t) = v_i(t), \quad \sigma(x_i-\Delta x/2, t) = \sigma_i
$$

と定義すれば，先の構成関係式と運動方程式は

$$
\begin{align}
	\frac{d \sigma_i(t)}{dt} = G\frac{v_{i}(t) - v_{i-1}(t) }{\Delta x}
	\\
	\rho \frac{d v_i(t)}{dt} = \frac{\sigma_{i+1}(t)- \sigma_i(t)}{\Delta x}
\end{align}
$$

となりました．変位速度と応力で添字$i$のつく場所を $\Delta x/2$ だけずらしたのがポイントです．そうでないと，$i\pm 1/2$という整数でない位置が現れてしまい，コンピュータプログラミング上不都合があるのです．グリッド番号 $i$ として整数だけを取ることとすると，これはもともとの運動方程式や構成関係式は任意の位置$x$における変位速度や応力を記述するのに対して，この式は$i=1, 2, \dots$の **離散的な** 位置における挙動のみを記述するように **離散化** した，ということに相当します．

続けて，時間微分についても同じように離散化してみます．構成関係式の左辺には応力の時間微分がありますが，応力が時間に対して連続であるという仮定のもと，これを適当な **時間ステップ** 間隔 $\Delta t$ を用いて

$$
\begin{align}
	\frac{d \sigma_i(t)}{d t} \simeq \frac{\sigma_i(t+\Delta t/2)-\sigma_i(t-\Delta t/2)}{\Delta t}
\end{align}
$$

と近似します．

求めたいのは時刻 $t$ における微分値なので，その時刻を中心とした $t \pm \Delta t/2$ の時刻における $v_i$ を用いて微分値を近似しました．
もとの構成関係式に代入して整理すると，

$$
\begin{align}
	\sigma_i(t+\Delta t/2) = \sigma_i(t-\Delta t/2)
	+ 
	G\frac{v_{i}(t) - v_{i-1} (t) }{\Delta x} \Delta t
\end{align}
$$

と書けます．
この式は，時刻 $t-\Delta t/2$ における同じ位置の応力と時刻 $t$ における $i$, $i+1$ 番目の変位速度から $t+\Delta t/2$ における変位速度を求める，という，過去から未来を予測するような式になっています．
もしさらに未来のことを求めたければ，$t\rightarrow t+\Delta t$ と全体の時間を1時間ステップだけ進めると，

$$
\begin{align}
	\sigma_i(t+3\Delta t/2) = \sigma_i(t+\Delta t/2)
	+ 
	G\frac{v_{i}(t+\Delta t) - v_{i-1}(t+\Delta t) }{\Delta x} \Delta t
\end{align}
$$

のようになります．地震波の数値シミュレーションは，このように過去の波動の状態から未来の状態を次々と予測することによってなされます．

続けて，変位速度について検討します．
構成関係式の時間についての離散化結果からは，変位速度 $v$ は時刻 $t$, $t+\Delta t$, $t+2\Delta t$, $\cdots$ における値がわかれば良さそうです．
そこで，運動方程式については構成関係式から $\Delta t/2$ だけ時間をずらして，

$$
\begin{align}
	\frac{\partial v_i(t+\Delta t/2)}{\partial t}
	\simeq 
	\frac{v_i(t+\Delta t) - v_i(t)}{\Delta t}
\end{align}
$$

とします．
すると，運動方程式は，

$$
\begin{align}
	v_i(t+\Delta t) = v_i(t) 
	+ 
	\frac{1}{\rho} 
	\frac{\sigma_{i+1}(t+\Delta t/2) - \sigma_{i}(t+\Delta t/2) }{\Delta x} \Delta t
\end{align}
$$

となります．

以上で，構成関係式と運動方程式ついて時間と空間の双方を離散化した式が得られました．
まとめると，

$$
\begin{align}
	 &	\sigma_i(t+\Delta t/2) = \sigma_i(t-\Delta t/2)
	 + 
	 G\frac{v_{i}(t) - v_{i-1} (t) }{\Delta x} \Delta t 
	\\
	 & v_i(t+\Delta t) = v_i(t) 
	 + 
	 \frac{1}{\rho} 
	 \frac{\sigma_{i+1}(t+\Delta t/2) - \sigma_{i}(t+\Delta t/2) }{\Delta x} \Delta t
\end{align}
$$

の2式です．

応力と変位速度の離散化された時刻が互いに $\Delta t/2$ だけずれていて，かつ両辺とも右辺の値が左辺よりも過去の情報だけで記述されています．しかも，構成関係式によって，あらゆる $i$ について応力 $\sigma_i(t+\Delta t/2)$ を求めておけば，それが運動方程式の右辺に使えます．すなわち，これらの式を交互に使うことで，変位速度と応力の時間発展を追跡できるのです．

具体的には，計算したい範囲 $i=1,\dots, n_x$ に対し，まず適当な初期条件 $\sigma_i(-\Delta t/2)$ と $v_i(0)$ を仮定します．
そして，

- 構成関係式から $\sigma_i(\Delta t/2)$ が求まる
- 運動方程式から $v_i(\Delta t)$ が求まる
- 構成関係式から $\sigma_i(3\Delta t/2)$ が求まる
- 運動方程式から $v_i(2 \Delta t)$ が求まる
- ・・・（以下繰り返し）

と繰り返すことで，逐次的に未来の状態をシミュレーションしていけるのです．

### 実践 1次元地震波数値シミュレーション

では，いきなりですが，ここまでで得られたアルゴリズムを踏まえて数値シミュレーションを実施してみましょう．

In [ ]:
dx = 0.2               # 空間グリッドサイズ (km)
dt = 0.05              # 時間ステップサイズ (s)
nx = 1001              # 空間グリッド数
nt = 401               # 時間ステップ数
L  = 8.0               # 初期条件の波長 (km)

V   = np.zeros(nx+1)   # 変位速度
S   = np.zeros(nx+2)   # 応力
rho = np.zeros(nx+1)   # 質量密度
G   = np.zeros(nx+1)   # 弾性係数

# 媒質の設定
beta   = 4.0               # 仮定する地震波速度 (km/s)
rho[:] = 2.7               # 仮定する質量密度 (g/cm^3)
G[:]   = rho * beta * beta # 剛性率 G = ρ x β^2

# 初期条件
hw = int(L/2/dx + 0.5)
for i in range(-hw, hw+1):
    V[int((nx - 1)/2)+1+i] = np.cos(np.pi/2.0*i/hw)**2

# 結果出力ファイルの準備と初期状態の出力
n = 0
fp = open('out.dat', 'w')
for i in range(1, nx+1):
    print(f"{(i-1)*dx:12.4e} {n*dt:12.4e} {V[i]:12.4e} {S[i]:12.4e}", 
          file=fp)

# 時間発展
for n in range(1, nt+1):
   
    # 構成関係式
    for i in range(1, nx+1):
        dxV = ( V[i] - V[i-1] ) / dx
        S[i] += G[i] * dxV * dt

    # 運動方程式
    for i in range(1, nx+1):
        dxS = ( S[i+1] - S[i] ) / dx
        V[i] += dxS / rho[i] * dt

    # データ出力
    for i in range(1, nx+1):
        print(f"{(i-1)*dx:12.4e} {n*dt:12.4e} {V[i]:12.4e} {S[i]:12.4e}", 
              file=fp)
    
fp.close()

コメントに「時間発展」とか書かれているところから先が差分法による数値シミュレーションのメイン部分です．数式通りに，まず右辺の空間微分を差分近似で評価し，時間差分を整理した結果をつかって左辺の値をアップーデートする，という流れで計算が進んでいることがわかるでしょうか．

上記コードを実行すると，`out.dat` というファイルが作られます．その中身の一部を取り出してみると，

```
  4.8200e+01   1.2800e+01   4.8168e-01   5.0092e+00
  4.8400e+01   1.2800e+01   4.9384e-01   5.2022e+00
  4.8600e+01   1.2800e+01   5.0000e-01   5.3335e+00
  4.8800e+01   1.2800e+01   5.0000e-01   5.4000e+00
  4.9000e+01   1.2800e+01   4.9384e-01   5.4000e+00
  4.9200e+01   1.2800e+01   4.8168e-01   5.3335e+00
```

のように4列からなるデータです．このデータは，左2列から位置 `x`, 時刻 `t`で，その位置と時間における変位速度 `V`, 応力 `S` の変数が3列目と4列目に格納されています．つまりこのデータ一つで，地震波の時間・空間両方の広がりが記述されていることになります．

いま興味があるのは位置・時間と変位速度ですので，結果ファイルからこれらを取り出します．

In [ ]:
d = np.loadtxt('out.dat') # 1行ずつのリストの集合体として読み込まれる
xx, tt, VV, _  = d.T      # 転置 `.T` により1列ごとのデータに変換する

このデータを可視化してみます．各地点における地震波形のレコードセクションを作成してみましょう．

In [ ]:
def wvplot_record(xx, tt, VV): 
    """シミュレーション結果のレコードセクション表示"""

    scale = 25
    
    fig = pygmt.Figure()

    fig.basemap(projection='X12c/8c', region=[0, 20, 0, 200], 
                frame = ['WS', 'xaf+ltime [s]', 'yaf+ldistance [km]'])
    
    for x in np.arange(0, 200, 2):

        # 位置 xx がループ変数 x に十分に近いところの時間・変位速度を取り出す
        mask = np.where(np.isclose(xx, x))
        ttt = tt[mask]
        VVV = VV[mask]

        # y軸位置を距離分だけずらしてプロットする
        fig.plot(x=ttt, y=x+VVV*scale, pen='thinner,black@50')

    return fig

In [ ]:
wvplot_record(xx, tt, VV)

続けて，振幅分布のアニメーションです．以下の関数 `wvplot_anim` は，時刻 0 から 20秒まで，0.2秒ずつの振幅分布をすべて作成します．それをパラパラマンガとして動画化してくれるのが，その次の `gif_movie` 関数です．

In [ ]:
def wvplot_anim(xx, tt, VV): 

    figs = []

    for t in np.arange(0, 20, 0.2):

        fig = pygmt.Figure()
        with pygmt.config(MAP_GRID_PEN_PRIMARY = '0.25p,30/30/30,.'): 
            fig.basemap(projection='X12c/6c', region=[0, 200, -0.1, 1.1], 
                        frame = ['WS', 
                                 'xafg+ldistance [km]', 
                                 'yafg+lvelocity amplitude'])
        mask = np.where(np.isclose(tt, t))
        xxx = xx[mask]
        VVV = VV[mask]
    
        fig.plot(x=xxx, y=VVV, pen='thicker,150/100/250')
        fig.text(x=198, y = 0.98, justify='RT', text=f"t = {t:5.1f} s", 
                 font='12p,Helvetica,Black')
        figs.append(fig)

    return figs

In [ ]:
def gif_movie(figs, dpi=720, crop='0.5c'): 
    
    """
    PyGMTのFigureオブジェクトのリストからGifアニメーションを作成する．Jupyter Notebook上で表示されるオブジェクトを返す．

    Parameters
    ----------
    figs : list of Figure
        PyGMTのFigureオブジェクトのリスト
    dpi : int, optional
        解像度 (default: 720)
    crop : str, optional
        余白のトリミング量 (default: '0.5c')

    Returns
    -------
    HTML : IPython.display.HTML
        Gifアニメーション
    """
    from IPython import display as dd
    import tempfile
    import base64
    import os
    
    with tempfile.TemporaryDirectory() as tmpdir:
        for i, fig in enumerate(figs):
            figname = f'plot_{i:05d}.png'
            print(f'\rsaving figs ... ({(i+1)/len(figs)*100:5.1f}%)', end='')
            fig.savefig(os.path.join(tmpdir, figname), dpi=dpi, crop=crop)
        print(' Done.')
        
        cmd1 = f'ffmpeg -i {tmpdir}/plot_%5d.png '
        cmd2 = f' -vf "scale=800:-1,split [a][b];[a] palettegen [p];[b][p] paletteuse" '
        cmd3 = f' {tmpdir}/out.gif > /dev/null 2>&1'
        print(f'making gif ... ', end='')
        os.system(cmd1 + cmd2 + cmd3)
        print(' Done.')
        
        with open(f'{tmpdir}/out.gif', 'rb') as f:
            b64 = base64.b64encode(f.read()).decode("ascii")
        
    return dd.HTML(f'<img src="data:image/gif;base64,{b64}" width="80%"/>')

In [ ]:
ff = wvplot_anim(xx, tt, VV)

In [ ]:
gif_movie(ff) #1分くらいかかります．辛抱強く待ちましょう．

これで一通りの1次元地震波伝播シミュレーションが出来上がりました．このシミュレーションは，`beta` や `rho` のパラメタを変えるだけで，不均質な媒質中の地震波伝播を再現できます．実際にパラメタを変えて，色々試してみましょう．